In [1]:
from linares_plot import *
import statsmodels.formula.api as smf

In [ ]:
df= pd.read_excel('C:\\Users\\David\\Desktop\\IDIBAPS\\simulations_radial\\results_simul_radial_linear_all.xlsx')

pal_ = sns.color_palette("Greens", n_colors=4)

sns.lineplot(x="position", y="interference", hue='delay_time', hue_order=[0, 1000, 2000, 3000],
             ci=95, palette=pal_, data=df)

plt.gca().spines['right'].set_visible(False)  # aesthetics                                                                              
plt.gca().spines['top'].set_visible(False)                                                                                  
plt.gca().get_xaxis().tick_bottom()                                                                                         
plt.gca().get_yaxis().tick_left()
plt.gca().tick_params(direction='in') #direction
plt.ylabel(r'attraction to fixation ($^\circ$)', fontsize=25);
plt.xlabel(r'eccentricity ($^\circ$)', fontsize=25);
plt.ylim(-2,12);
plt.yticks([0,4,8,12]);
plt.plot([60, 300], [0,0], 'k--', linewidth=1)

l =plt.legend(loc=2, frameon=False, prop={'size': 15}, ncol=1)

hue_order=['', 'delay=0s', 'delay=1s', 'delay=2s', 'delay=3s']
palette=['black', pal_[0], pal_[1], pal_[2], pal_[3]]

for i_h, h_idx in enumerate(hue_order):
    l.get_texts()[i_h].set_text(h_idx)
    l.legendHandles[i_h].set_visible(False);
    l.get_texts()[i_h].set_color(palette[i_h]);

In [44]:
angles = [[20, 120, 300, 16, 160, 274], [20, 80, 300, 315, 270, 290]]
d = pd.DataFrame(angles)
d.columns=['T', 'NT1', 'NT2', 'Dist', 'Dist_NT1', 'Dist_NT2']
d

,T,NT1,NT2,Dist,Dist_NT1,Dist_NT2
0,20,120,300,16,160,274
1,20,80,300,315,270,290


In [45]:
def err_deg(a1,ref):
    ### Calculate the error ref-a1 in an efficient way in the circular space
    ### it uses complex numbers!
    ### Input in degrees (0-360)
    a1=np.radians(a1)
    ref=np.radians(ref)
    err = np.angle(np.exp(1j*ref)/np.exp(1j*(a1) ), deg=True) 
    err=round(err, 2)
    return err

In [46]:
close_one=[]

for i in range(len(d)):
    err1 = abs(err_deg(d['T'].iloc[i], d['Dist'].iloc[i]))
    err2 =abs(err_deg(d['NT1'].iloc[i], d['Dist_NT1'].iloc[i]))
    err3 =abs(err_deg(d['NT2'].iloc[i], d['Dist_NT2'].iloc[i]))

    options = ['T', 'NT1', 'NT2']
    erros_dist = [err1, err2, err3]
    pos_min_err = np.where(erros_dist==min(erros_dist))[0][0]
    close_one.append( d[options[pos_min_err]].iloc[i] )

In [47]:
close_one

[20, 300]